# Instagram API testing

### Data we need to retrieve given a user profile

1. Number of followers
2. Number of posts

For each post:

3. Number of comments
4. Number of likes
5. Post caption
6. Post comments text

Gather all the data for future SPI computation

The API has some limitations:
* There is a rate limit of 200req/hour. Going beyond will break the Instagram Private API (i.e. will block our account)

It would be nice to create several IG accounts and parallelize the job. That way, the job will be done more efficiently and we will
not expose ourselves to be blocked that much.

In [4]:
n_posts = 5
print("Number of users processed by minute: {0:1f} in order not to be blocked by IG".format(200/((3+n_posts)*60)))
print("Delay between users information retrieval: {0:1f} min in order not to be blocked by IG".format(1/(200/((3+n_posts)*60))))

Number of users processed by minute: 0.416667 in order not to be blocked by IG
Delay between users information retrieval: 2.400000 min in order not to be blocked by IG


In [5]:
from instagrapi import Client
from datetime import date

import time
import numpy as np

cl = Client()
# cl.login("clyde_donovan_2022", "clyde_donovan_2022_")
# cl.login("chrismiller202212", "1OE0t5@Po9*z_")
cl.login("samanthaoakley202212", "1OE0t5@Po9*z")
user_name = "shakira"
max_posts = 5

user_id = cl.user_id_from_username(user_name)
posts = cl.user_medias(user_id, max_posts)
user_information = cl.user_info(user_id)

# time.sleep(60)

1. Get the number of followers and 2. the number of posts

In [3]:
n_followers = user_information.follower_count
n_following = user_information.following_count
n_posts = user_information.media_count

In [4]:
n_posts

2034

For each post get the 3. number of comments, 4. number of likes, 5. caption, 6. comments text

In [5]:
def extract_text(comment):
    comments_text_ls = []
    if len(comment[0]) != 0:
        comments_text_ls = [comment.text for comment in comment[0]]
    return comments_text_ls

In [6]:
# Looks like max_comments does nothing in media_comments_chunk()
max_comments = 5

posts_info = {}
for i, post in enumerate(posts):
    if i > 9:
        posts_info[post.id] = {
                              "n_comments": post.comment_count, 
                              "n_likes": post.like_count, 
                              "caption": post.caption_text, 
                              "comments_text": []
                            }    
    else: 
      posts_info[post.id] = {
                              "n_comments": post.comment_count, 
                              "n_likes": post.like_count, 
                              "caption": post.caption_text, 
                              "comments_text": extract_text(cl.media_comments_chunk(post.id, max_amount=max_comments))
                            }                        

Gather all the data for future SPI computation

In [7]:
user_1_ig_data = {
    "user_name": user_name,
    "user_id": user_id,
    "n_followers": n_followers,
    "n_following": n_following,
    "n_posts_total": n_posts,
    "n_posts_retrieved": len(posts_info),
    "n_likes_total": np.sum([posts_info[key]["n_likes"] for key in posts_info.keys()]),
    "n_comments_total": np.sum([posts_info[key]["n_comments"] for key in posts_info.keys()]),
    "n_comments_retrieved": np.sum([len(posts_info[key]["comments_text"]) for key in posts_info.keys()]),
    "date": date.today().strftime("%d/%m/%Y"),
    "posts_info": posts_info,
}

In [8]:
user_1_ig_data

{'user_name': 'shakira',
 'user_id': '217867189',
 'n_followers': 77482794,
 'n_following': 152,
 'n_posts_total': 2034,
 'n_posts_retrieved': 2034,
 'n_likes_total': 990972238,
 'n_comments_total': 9027552,
 'n_comments_retrieved': 382,
 'date': '27/11/2022',
 'posts_info': {'2980186703258291460_217867189': {'n_comments': 8331,
   'n_likes': 695029,
   'caption': 'Remembering World Cup Magic #WorldCup',
   'comments_text': ['@aduncangb ere en tu ???',
    '😍😍😍😍😍😍😍',
    '@thatjltguy 👎',
    '@utophya_ 👎',
    '@espeeranci pues ya ves que si hay !!! No sabía que ella lo organizaba😂😂gente brutaaa!!!',
    'Waka Waka #1, Copa de la Vida #2, We Are One #3',
    '❤️🔥❤️👏👏🏾👏🙏⛱🤔..',
    '❤️',
    '🔥',
    'Eres el mejor emblema de los mundiales, te queremos 😍🔥👏👏👏',
    'Mi bella shakira en ves de estar escriendole canción al bobo ese poque no creaste otra obra maestra de las tuya pa este mundial!',
    'Por complacer a todo sus seguidores debe ir ala clausura 👏',
    'سلاااااااااام عليكم',
  

In [10]:
user_name = "jfcadavid"
max_posts = 5

user_id = cl.user_id_from_username(user_name)
posts = cl.user_medias(user_id, max_posts)
user_information = cl.user_info(user_id)

# time.sleep(60)